In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import math
import os


import tensorflow as tf

import configuration
import inference_wrapper
from inference_utils import caption_generator
from inference_utils import vocabulary

C:\Users\W2W\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
FLAGS = tf.flags.FLAGS

In [5]:
image_name = input("Enter name of image (e.g. 2.jpg):")

Enter name of image (e.g. 2.jpg):2.jpg


In [6]:
tf.flags.DEFINE_string("input_files", "./test_images/" + image_name,
                       "File pattern or comma-separated list of file patterns "
                       "of image files.")

In [7]:
tf.flags.DEFINE_string("checkpoint_path", "./model.ckpt-2000000",
                       "Model checkpoint file or directory containing a "
                       "model checkpoint file.")
tf.flags.DEFINE_string("vocab_file", "./word_counts.txt", "Text file containing the vocabulary.")
tf.logging.set_verbosity(tf.logging.INFO)


In [8]:
def main(_):
  # Build the inference graph.
  g = tf.Graph()
  with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(),
                                               FLAGS.checkpoint_path)
  g.finalize()

  # Create the vocabulary.
  vocab = vocabulary.Vocabulary(FLAGS.vocab_file)

  filenames = []
  for file_pattern in FLAGS.input_files.split(","):
    filenames.extend(tf.gfile.Glob(file_pattern))
  tf.logging.info("Running caption generation on %d files matching %s",
                  len(filenames), FLAGS.input_files)

  with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)

    # Prepare the caption generator. Here we are implicitly using the default
    # beam search parameters. See caption_generator.py for a description of the
    # available beam search parameters.
    generator = caption_generator.CaptionGenerator(model, vocab)

    for filename in filenames:
      with tf.gfile.GFile(filename, "rb") as f:
        image = f.read()
      captions = generator.beam_search(sess, image)
      print("Captions for image %s:" % os.path.basename(filename))
      for i, caption in enumerate(captions):
        # Ignore begin and end words.
        sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
        sentence = " ".join(sentence)
        print("  %d) %s (p=%f)" % (i, sentence, math.exp(caption.logprob)))

In [9]:
if __name__ == "__main__":
  tf.app.run()

INFO:tensorflow:Building model.
INFO:tensorflow:Initializing vocabulary from file: ./word_counts.txt
INFO:tensorflow:Created vocabulary with 11520 words
INFO:tensorflow:Running caption generation on 1 files matching ./test_images/2.jpg
INFO:tensorflow:Loading model from checkpoint: ./model.ckpt-2000000
INFO:tensorflow:Restoring parameters from ./model.ckpt-2000000
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-2000000
Captions for image 2.jpg:
  0) a man riding a paddle board on top of a river . (p=0.000126)
  1) a man riding a paddle board on a river . (p=0.000061)
  2) a man riding a paddle board on top of a lake . (p=0.000022)


SystemExit: 

C:\Users\W2W\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
